In [16]:
from MyQR import myqr 
import random

import PySimpleGUI as sg


class ticket:
    def __init__(self,productes,accions,punts_total=0,preu_total=0):
        #llista de productes
        self.productes=productes
        #llista d'accions
        self.accions=accions
        #total de punts acumulats en la compra
        self.punts_total=punts_total
        #preu de la compra
        self.preu_total=preu_total
            
        for element in productes.llista_productes:
            self.preu_total+=(element.preu)*element.unitats
            self.punts_total+=(element.punts)*element.unitats
            
        punts_per_accio={'no_bossa_plastic':2,'retornar_ampolla_vidre':3}
        for accio in accions:
            self.punts_total+=punts_per_accio[accio]
        
        print('Punts total: ',self.punts_total,'\nPreu total: ',self.preu_total) 
        
    def get_qr(self):
        #generem número aleatori per l'ID, el multipliquem per 1000 i afegim la puntuacio
        ident=random.randint(0,1000000000)
        num=ident*1000+self.punts_total
        myqr.run(words=str(num),version=1,
        save_name="qr.png",
        save_dir="C:\Reproductor")

class producte:
    def __init__(self,unitats,nom,preu,no_plastic_porexpan,temporada,local,punts=0):
        self.unitats=unitats
        self.nom=nom
        self.preu=preu
        #0 si plàstic o porexpan , 1 no plàstic no porexpan
        self.no_plastic_porexpan=no_plastic_porexpan
        #0 no és de temporada, 1 si és de temporada  
        self.temporada=temporada
        #0 no és producte local, 1 si és local
        self.local=local
        
        self.punts=punts
        if self.no_plastic_porexpan==True:
            self.punts+=1
        if self.temporada==True:
            self.punts+=1
        if self.local==True:
            self.punts+=1

class llista_productes:
    def __init__(self):
        self.llista_productes=list()    
        
    def afegir_producte(self,producte):    
        self.llista_productes.append(producte)
    
    def eliminar_producte(self,nom_producte):
        for producte in self.llista_productes:
            if nom_producte==producte.nom:
                self.llista_productes.remove(producte)
        
    def consultar_prod(self,n):
        return llista_productes[n]
        

        
#"-------------------------------------------------------------------------"
file_list_column = [[sg.Listbox(values=[], enable_events=True, size=(40, 20), key="listbox")]]

layout1 = [[sg.Text("NOMBRE DE LA APP")],
          [sg.Text()],
          [sg.Text()],
          [sg.Text()],
          [sg.Text("Nom:    "),sg.In(size=(25, 1), enable_events=True, key="nom")], 
          [sg.Text("Preu:    "),sg.In(size=(25, 1), enable_events=True, key="preu")],
          [sg.Text("Unitats: "),sg.In(size=(25, 1), enable_events=True, key="unitats")],
          [sg.Text()],
          [sg.Checkbox('És de temporada',key='temporada',enable_events=True)],
          [sg.Checkbox('No és de plàstic ni porexpan',key='plastic_porexpan',enable_events=True)],
          [sg.Checkbox('És producte local',key='local',enable_events=True)],
          [sg.Text()],
          [sg.Button("AFEGIR PRODUCTE")],
          [sg.Button("ELIMINAR PRODUCTE")]
         ]

layout2=[[sg.Text("PRODUCTES")],
         [sg.Column(file_list_column)],
         [sg.Checkbox('No compra bossa plàstic',key='bossa_plastic',enable_events=True)],
         [sg.Checkbox('Retorna envasos',key='retorna_envasos',enable_events=True)],
         [sg.Button("EXTREURE QR")]
        ]


layout = [[
        sg.Column(layout1),
        sg.VSeperator(),
        sg.Column(layout2),
    ]]


# Create the window
window = sg.Window("Hackathon", layout)


productes=llista_productes()
accions=list()
llista_noms=list()

# Create an event loop
while True:
    event, values = window.read()
    
    if event == "AFEGIR PRODUCTE":
        p=producte(int(values["unitats"]),values["nom"],float(values["preu"]),values["plastic_porexpan"],values["temporada"],values["local"])  
        productes.afegir_producte(p)
        llista_noms.append(values["nom"])
        window["listbox"].update(llista_noms)
        
    if event == "ELIMINAR PRODUCTE":
        productes.eliminar_producte(values['listbox'][0])
        llista_noms.remove(values['listbox'][0])
        window["listbox"].update(llista_noms)
        

    if event == "EXTREURE QR":
        if values['bossa_plastic']:
            accions.append('no_bossa_plastic')
        if values['retorna_envasos']:
            accions.append('retornar_ampolla_vidre')
            
        ticket_compra=ticket(productes,accions)
        ticket_compra.get_qr()
        
    
        sg.popup(image='C:\Reproductor\qr.png')
        
        
    if event == sg.WIN_CLOSED:
        break
       
        
window.close()